![Podmanlogo](Pictures/podman-logo.png)

# Security with Podman rootless containers

Podman is a container engine designed to work with rootless containers, meaning that there is no need of using sudo to run your containers. If you have used other container engines you may have seen the need of running your container engine using sudo or adding your non-root user to a group with priviledges. This is, of course, a concern for the security of your environment.

Running a container engine with the root user exposes two important methods of attacking your system:

 - If your container engine has a vulnerability your entire system will be compromised as the process of your container engine would be owned by the root user.
 - Given a container image that contains a vulnerability an attacker could gain access to your vulnerable image and, as your container image is being executed by the root user, the attacker could gain access to the whole system from it.

Podman has a simple solution for this: don't run everything as root. By default Podman does not use the root user, but it can be used for those edge use cases in which it's needed. This is what is called a zero-trust approach, meaning that Podman wont be using the root user unless it's specified.

Running containers with non-root users also gives you the hability of isolate group of containers being run by different users in the same system. Lets say you have Application A composed of three containers (frontend, backend and database) and Application B composed of two containers (frontend and backend), the applications do not interact between themselves, so you would like to isolate them. In this scenario you can just create two users (application-a-user and application-b-user) and instanciate each application with its own user. Once you do that, if for example Application B is attacked via a vulnerability, only the containers and processes run by the application-user-b user would be exposed.

Of course, there are some scenarios in which running rootless containers could limit functionality. Lets review the most common caveats of rootless containers:

 - Exposing your container in ports below 1024 cannot be binded out of the box. There are workarounds for this.
 - Sharing images between users needs to be done via [this process](https://www.redhat.com/sysadmin/podman-transfer-container-images-without-registry?ref=linuxhandbook.com)

## Rootless containers with podman

First thing we need to understand is that in a linux system each user will have a list of subuid and subgid's that can be used for running processes. These can be found in the files /etc/subuid and /etc/subgid.

In [13]:
!cat /etc/subuid

ppreciad:100000:65536


In [14]:
!cat /etc/subgid

ppreciad:100000:65536


In the output of those two files you see the range of the subuid and subgid that can be used for executing processes by your user. This values can be modified with the usermod command, but we will not be covering it in this workshop.

Lets show rootless containers working, first we will execute a new container:

In [15]:
!podman run -d --rm --name=rootless-container docker.io/redhat/ubi9 sleep 999

c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943


Check that your container is running

In [16]:
!podman ps

CONTAINER ID  IMAGE                         COMMAND     CREATED        STATUS        PORTS       NAMES
c7b48293382a  docker.io/redhat/ubi9:latest  sleep 999   4 seconds ago  Up 5 seconds              rootless-container


Now use the "ps aux" command to see the uid of the user that executed our container.

In [17]:
!echo "USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND"
!ps aux | grep rootless


USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
ppreciad   69697  0.0  0.0 225852  1928 ?        Ss   18:11   0:00 /usr/bin/conmon --api-version 1 -c c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943 -u c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943 -r /usr/bin/crun -b /home/ppreciad/.local/share/containers/storage/overlay-containers/c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943/userdata -p /run/user/1000/containers/overlay-containers/c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943/userdata/pidfile -n rootless-container --exit-dir /run/user/1000/libpod/tmp/exits --full-attach -s -l journald --log-level warning --syslog --runtime-arg --log-format=json --runtime-arg --log --runtime-arg=/run/user/1000/containers/overlay-containers/c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943/userdata/oci-log --conmon-pidfile /run/user/1000/containers/overlay-containers/c7b48293382afc0201b606

As you can see the output shows the container process using a non priviledged UID.

Last point I would like to highlight regarding security is that we never used sudo for running podman. In general we have not used root user in any moment of this workshop, following a zero-trust security approach.

# Cleanup

In [18]:
!podman stop --all
!podman rm --all
!podman rmi docker.io/redhat/ubi9

WARN[0010] StopSignal SIGTERM failed to stop container rootless-container in 10 seconds, resorting to SIGKILL 
WARN[0010] StopSignal SIGTERM failed to stop container kubepodman in 10 seconds, resorting to SIGKILL 
c7b48293382afc0201b606835b91f15e5f4623d300395799edc65dcc550df943
d696dccd5291f64879789353dfb858d9511c6ecf133215148f9bf04a767cfa41
Untagged: docker.io/redhat/ubi9:latest
